#We combine the features extracted from pre-trained BERT humor detection and sentiment analysis models and train them on our data for the tri-classification task

We have total 1024 features/sample, 768 from sentiment model and 256 from humor model

In [1]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/meme_classification_data/train.csv')
df_train.head(3)

,ID,image id,text,label,label_num
0,1,image_2455.jpg,- It is not our fight - Are we not part of thi...,troll,2
1,2,image_3701.jpg,THAT'S THE DIFFERENCE BETWEEN YOU AND ME YOU...,none,0
2,3,image_4166.png,- WHAT DO THE TITANIC AND THE SIXTH SENSE HAVE...,none,0


#Loading Features and combining

In [2]:
import numpy as np
df_sent = pd.read_csv('/content/drive/MyDrive/meme_classification_data_MMN/train_Sentiment.csv')
df_humor = pd.read_csv('/content/drive/MyDrive/meme_classification_data_MMN/train_Humor.csv')
X_sent = df_sent.values
X_humor = df_humor.values
X = np.concatenate((X_sent, X_humor), axis = 1)
y = df_train['label_num']
X.shape, y.shape

((1991, 1024), (1991,))

#Training NN on the features


In [3]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical   
categorical_labels = to_categorical(df_train['label_num'], num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, categorical_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((1393, 1024), (199, 1024), (399, 1024), (1393, 3), (199, 3), (399, 3))

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop,Adam
model=Sequential() 
model.add(Dense(50,activation='sigmoid',input_shape=(1024,),kernel_initializer='he_normal'))
model.add(Dropout(0.2))
#model.add(Dense(20,activation='sigmoid'))
model.add(Dense(5,activation='sigmoid'))
model.add(Dense(3,activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                51250     
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        
Total params: 51,523
Trainable params: 51,523
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy','AUC'])


history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=50,
                    verbose=1,
                    validation_data=(X_val, y_val))


Epoch 1/50
44/44 [==============================] - 1s 10ms/step - loss: 1.1023 - accuracy: 0.3628 - auc: 0.5247 - val_loss: 1.1073 - val_accuracy: 0.3216 - val_auc: 0.5067
Epoch 2/50
44/44 [==============================] - 0s 3ms/step - loss: 1.0947 - accuracy: 0.3633 - auc: 0.5431 - val_loss: 1.0963 - val_accuracy: 0.3668 - val_auc: 0.5380
Epoch 3/50
44/44 [==============================] - 0s 4ms/step - loss: 1.0833 - accuracy: 0.3835 - auc: 0.5728 - val_loss: 1.0952 - val_accuracy: 0.4121 - val_auc: 0.5566
Epoch 4/50
44/44 [==============================] - 0s 3ms/step - loss: 1.0812 - accuracy: 0.3883 - auc: 0.5811 - val_loss: 1.1067 - val_accuracy: 0.3166 - val_auc: 0.5195
Epoch 5/50
44/44 [==============================] - 0s 4ms/step - loss: 1.0721 - accuracy: 0.4130 - auc: 0.5960 - val_loss: 1.0864 - val_accuracy: 0.4121 - val_auc: 0.5743
Epoch 6/50
44/44 [==============================] - 0s 4ms/step - loss: 1.0662 - accuracy: 0.4328 - auc: 0.6142 - val_loss: 1.1008 - val_ac

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred = np.argmax(y_pred, axis = 1)
y_true = np.argmax(y_test, axis = 1)
print(classification_report(y_true,y_pred))

7/7 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.36      0.38      0.37       117
           1       0.43      0.32      0.37       136
           2       0.44      0.53      0.48       146

    accuracy                           0.41       399
   macro avg       0.41      0.41      0.41       399
weighted avg       0.41      0.41      0.41       399



#Training on SVC, Tree Based Classifiers and Gradient Boost Classifiers

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
models=[SVC(kernel='rbf'), SVC(kernel = 'linear'), SVC(kernel = 'poly', degree = 6),RandomForestClassifier(),DecisionTreeClassifier(), XGBClassifier()]
names = ['SVM-rbf','SVM-linear','SVM-poly','Random-Forest','Decision-Tree', 'XGB']
for i in range(len(models)):
  models[i].fit(X_train,y_train)
  print("MODEL: ", names[i])
  print("On Train")
  print(classification_report(y_train,models[i].predict(X_train)))
  print("On Test")
  print(classification_report(y_test, models[i].predict(X_test)))

MODEL:  SVM-rbf
On Train
              precision    recall  f1-score   support

           0       0.56      0.28      0.37       487
           1       0.50      0.58      0.54       553
           2       0.47      0.61      0.53       552

    accuracy                           0.50      1592
   macro avg       0.51      0.49      0.48      1592
weighted avg       0.51      0.50      0.48      1592

On Test
              precision    recall  f1-score   support

           0       0.36      0.17      0.23       117
           1       0.39      0.47      0.43       136
           2       0.43      0.53      0.48       146

    accuracy                           0.41       399
   macro avg       0.40      0.39      0.38       399
weighted avg       0.40      0.41      0.39       399

MODEL:  SVM-linear
On Train
              precision    recall  f1-score   support

           0       0.84      0.85      0.85       487
           1       0.86      0.85      0.85       553
           2  

#PCA and trying models again

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=500)
pca.fit(X)
X = pca.transform(X)
X.shape

(1991, 500)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
models=[SVC(kernel='rbf'), SVC(kernel = 'linear'), SVC(kernel = 'poly', degree = 6),RandomForestClassifier(),DecisionTreeClassifier(), XGBClassifier()]
names = ['SVM-rbf','SVM-linear','SVM-poly','Random-Forest','Decision-Tree', 'XGB']
for i in range(len(models)):
  models[i].fit(X_train,y_train)
  print("MODEL: ", names[i])
  print("On Train")
  print(classification_report(y_train,models[i].predict(X_train)))
  print("On Test")
  print(classification_report(y_test, models[i].predict(X_test)))

MODEL:  SVM-rbf
On Train
              precision    recall  f1-score   support

           0       0.75      0.49      0.59       487
           1       0.62      0.70      0.66       553
           2       0.59      0.68      0.63       552

    accuracy                           0.63      1592
   macro avg       0.65      0.63      0.63      1592
weighted avg       0.65      0.63      0.63      1592

On Test
              precision    recall  f1-score   support

           0       0.42      0.25      0.31       117
           1       0.37      0.46      0.41       136
           2       0.43      0.48      0.45       146

    accuracy                           0.40       399
   macro avg       0.41      0.39      0.39       399
weighted avg       0.41      0.40      0.40       399

MODEL:  SVM-linear
On Train
              precision    recall  f1-score   support

           0       0.74      0.77      0.75       487
           1       0.80      0.75      0.77       553
           2  

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical   
categorical_labels = to_categorical(df_train['label_num'], num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, categorical_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((1393, 500), (199, 500), (399, 500), (1393, 3), (199, 3), (399, 3))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop,Adam
model=Sequential() 
model.add(Dense(50,activation='sigmoid',input_shape=(500,),kernel_initializer='he_normal'))
model.add(Dropout(0.1))
#model.add(Dense(20,activation='sigmoid'))
model.add(Dense(5,activation='sigmoid'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 50)                25050     
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 5)                 255       
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 18        
Total params: 25,323
Trainable params: 25,323
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy','AUC'])


history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=25,
                    verbose=1,
                    validation_data=(X_val, y_val))


Epoch 1/25
22/22 [==============================] - 1s 21ms/step - loss: 1.1494 - accuracy: 0.2976 - auc: 0.4740 - val_loss: 1.1008 - val_accuracy: 0.2915 - val_auc: 0.5148
Epoch 2/25
22/22 [==============================] - 0s 6ms/step - loss: 1.0951 - accuracy: 0.3512 - auc: 0.5309 - val_loss: 1.0937 - val_accuracy: 0.4171 - val_auc: 0.5454
Epoch 3/25
22/22 [==============================] - 0s 5ms/step - loss: 1.0885 - accuracy: 0.4017 - auc: 0.5747 - val_loss: 1.0907 - val_accuracy: 0.4020 - val_auc: 0.5626
Epoch 4/25
22/22 [==============================] - 0s 6ms/step - loss: 1.0743 - accuracy: 0.4465 - auc: 0.6339 - val_loss: 1.0886 - val_accuracy: 0.4020 - val_auc: 0.5690
Epoch 5/25
22/22 [==============================] - 0s 6ms/step - loss: 1.0731 - accuracy: 0.4470 - auc: 0.6273 - val_loss: 1.0871 - val_accuracy: 0.3970 - val_auc: 0.5713
Epoch 6/25
22/22 [==============================] - 0s 4ms/step - loss: 1.0661 - accuracy: 0.4582 - auc: 0.6453 - val_loss: 1.0873 - val_ac

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred = np.argmax(y_pred, axis = 1)
y_true = np.argmax(y_test, axis = 1)
print(classification_report(y_true,y_pred))

7/7 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.37      0.30      0.33       117
           1       0.42      0.40      0.41       136
           2       0.45      0.54      0.49       146

    accuracy                           0.42       399
   macro avg       0.41      0.41      0.41       399
weighted avg       0.42      0.42      0.42       399

